In [ ]:
import shots_data_retriever
import importlib

importlib.reload(shots_data_retriever)

<module 'shots_data_retriever' from 'c:\\Users\\Dell.DESKTOP-4AU2O53\\Documents\\UdeM\\IFT 6758\\Projet\\ift6758\\visualizations\\shots_data_retriever.py'>

In [ ]:
shotsDataRetriever = shots_data_retriever.ShotsDataRetriever()

df = shotsDataRetriever.get_df_for_milestone2_part2()

print(df)

       is_goal  is_empty_net   distance  angle_to_goal
0            0             0  13.928388      21.037511
1            0             0  13.601471      72.897271
2            0             0  77.025970     -29.560354
3            0             0  58.940648     -14.743563
4            0             0  62.609903      26.565051
...        ...           ...        ...            ...
68495        0             0  26.419690     -60.524111
68496        0             0  40.496913      69.775141
68497        0             0  14.866069      42.273689
68498        0             0  11.401754     -37.874984
68499        0             0  26.925824     -31.328693

[305814 rows x 4 columns]
